# This code is for scraping only company name, job title, location, salary, and the URL for where the job description will be available


In [1]:
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement



In [6]:
def extract(page):
    url=f'https://www.indeed.com/jobs?q=data%20scientist&l=United%20States&start={page}'
    driver = webdriver.Chrome(executable_path='/Users/raminstad/Downloads/chromedriver')
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

def transform(soup):
    links=[]
    sleep(5)
    for i in soup.find_all('tbody'):

        l = i.find('tr')

        for j in l.find_all('a', href=True):
            if j.find('a', href=True):
                links.append(j['href'])
                link = j['href']
                divs = j.find_all('div', class_='slider_container')
                for g in divs:
                    title = g.find('table').text.strip()
                    company_name = g.find('span', class_='companyName').text
                    location = g.find('div', class_='companyLocation').text
                    try:
                        salary = g.find('div', class_='attribute_snippet').text
                    except:
                        salary = 'not_listed'

                    job = {
                        'Title': title,
                        'Company': company_name,
                        'Location': location,
                        'Salary': salary,
                        'URL': link
                    }
                    jobs.append(job)
    return


jobs=[]

# scraping 66 pages!
for i in range(0,661,10):
    c=extract(i)
    transform(c)

df=pd.DataFrame(jobs)
df.to_pickle('indeed.pkl')
df=pd.read_pickle('indeed.pkl')
df.reset_index(inplace=True)
df.drop(columns=['index'],inplace=True)

df.head()



,Title,Company,Location,Salary,URL
0,"Data ScientistTikTok3.5Mountain View, CA+27 lo...",TikTok,"Mountain View, CA+27 locations",not_listed,/pagead/clk?mo=r&ad=-6NYlbfkN0A1xek5knTnC5i_gS...
1,newClinical Studies Data ScientistTailored Man...,Tailored Management,Remote,$54.34 an hour,/pagead/clk?mo=r&ad=-6NYlbfkN0DI_pqscLjs9LkB0j...
2,newData Scientist or BioinformaticianUT Southw...,UT Southwestern Medical Center,+4 locationsRemote,"$60,000 - $100,000 a year",/company/UT-Southwestern-Medical-Center/jobs/D...
3,Data Scientist (Entry Level- Remote)HiLabsRale...,HiLabs,"Raleigh-Durham, NC+1 location•Remote","$120,000 - $150,000 a year",/company/HILABS/jobs/Data-Scientist-bbbc165952...
4,newData Scientist - Full RemoteVaco3.7+1 locat...,Vaco,+1 locationRemote,"$145,000 - $170,000 a year",/pagead/clk?mo=r&ad=-6NYlbfkN0D_sybMACCpf9B-67...


# This code is for going to each URL for every job posting in the data frame and scrape the Job description

In [7]:
# Adding 'http://www.indeed.com' to the beginig of URL column and reseting the index of the dataframe
df['URL']=df['URL'].apply(lambda x: 'http://www.indeed.com'+x)
df.reset_index(inplace=True)


def urlget(url):
    """
    This Function takes url as an argument and scrapes the job description
    """
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path='/Users/raminstad/Downloads/chromedriver',chrome_options=chrome_options)
    driver.get(url)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    divs = soup.find_all('div', class_='jobsearch-jobDescriptionText')
    sleep(randint(6,10))
    for content in divs:
        return content
        
    driver.quit()


# job_summary is the new column in the dataframe and the function above goes through every
# url in the dataframe and scrapes the job description
df['job_summary']=df['URL'].apply(lambda x: urlget(x))


df.head()



,Title,Company,Location,Salary,URL,job_summary
0,"Data ScientistTikTok3.5Mountain View, CA+27 lo...",TikTok,"Mountain View, CA+27 locations",not_listed,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
1,newClinical Studies Data ScientistTailored Man...,Tailored Management,Remote,$54.34 an hour,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
2,newData Scientist or BioinformaticianUT Southw...,UT Southwestern Medical Center,+4 locationsRemote,"$60,000 - $100,000 a year",http://www.indeed.com/company/UT-Southwestern-...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
3,Data Scientist (Entry Level- Remote)HiLabsRale...,HiLabs,"Raleigh-Durham, NC+1 location•Remote","$120,000 - $150,000 a year",http://www.indeed.com/company/HILABS/jobs/Data...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
4,newData Scientist - Full RemoteVaco3.7+1 locat...,Vaco,+1 locationRemote,"$145,000 - $170,000 a year",http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
